In [3]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,LabelEncoder,OneHotEncoder
import pickle

In [4]:
data=pd.read_csv('Churn_Modelling.csv')
data.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [5]:
data=data.drop(['RowNumber','CustomerId','Surname'],axis=1)

In [9]:
label_enco_gender=LabelEncoder()
data['Gender']=label_enco_gender.fit_transform(data['Gender'])

In [ ]:
ohe_geo=OneHotEncoder()
enco_geo=ohe_geo.fit_transform(data[['Geography']]).toarray()


In [21]:
enco_geo_df=pd.DataFrame(enco_geo,columns=ohe_geo.get_feature_names_out(['Geography']))

In [23]:
data=data.drop(['Geography'],axis=1)

In [24]:
data=pd.concat([data,enco_geo_df],axis=1)

In [25]:
data.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0


In [26]:
x=data.drop(['EstimatedSalary'],axis=1)
y=data['EstimatedSalary']

In [27]:
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=42)

In [28]:
standard_scaler=StandardScaler()
x_train=standard_scaler.fit_transform(x_train)
x_test=standard_scaler.transform(x_test)

ANN regression pr statement

In [30]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout

In [32]:
model=Sequential([
    Dense(64,activation='relu',input_shape=(x_train.shape[1],)),
    Dense(32,activation='relu'),
    Dense(1)
])

# compile the model

model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae'])
model.summary()


Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_3 (Dense)             (None, 64)                832       
                                                                 
 dense_4 (Dense)             (None, 32)                2080      
                                                                 
 dense_5 (Dense)             (None, 1)                 33        
                                                                 
Total params: 2945 (11.50 KB)
Trainable params: 2945 (11.50 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [33]:
# setup tensorboard callback
import datetime
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard

log_dir = "Regressionlogs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)

In [34]:
# setup early stopping
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

In [35]:
history=model.fit(
    x_train,y_train,
    validation_data=(x_test, y_test),
    epochs=100,
    callbacks=[early_stopping, tensorboard_callback]
)

Epoch 1/100


250/250 [==============================] - 6s 9ms/step - loss: 13385897984.0000 - mae: 100388.3438 - val_loss: 13015080960.0000 - val_mae: 98558.7969
Epoch 2/100
250/250 [==============================] - 1s 6ms/step - loss: 13263801344.0000 - mae: 99780.7266 - val_loss: 12772831232.0000 - val_mae: 97332.7109
Epoch 3/100
250/250 [==============================] - 2s 7ms/step - loss: 12830872576.0000 - mae: 97631.3750 - val_loss: 12148534272.0000 - val_mae: 94175.9141
Epoch 4/100
250/250 [==============================] - 2s 7ms/step - loss: 11958817792.0000 - mae: 93310.4688 - val_loss: 11072243712.0000 - val_mae: 88732.6797
Epoch 5/100
250/250 [==============================] - 3s 12ms/step - loss: 10644623360.0000 - mae: 86790.3594 - val_loss: 9611583488.0000 - val_mae: 81364.8750
Epoch 6/100
250/250 [==============================] - 2s 8ms/step - loss: 9022588928.0000 - mae: 78673.4062 - val_loss: 7959169024.0000 - val_mae: 73044.3047
Epoch 7/100
250/250 [============

In [36]:
%load_ext tensorboard

In [37]:
%tensorboard --logdir Regressionlogs/fit

In [38]:
# evaluate for test data
test_loss, test_mae = model.evaluate(x_test, y_test, verbose=2)
print(f"Test Loss: {test_loss}, Test MAE: {test_mae}")

63/63 - 0s - loss: 3354440448.0000 - mae: 50089.4258 - 241ms/epoch - 4ms/step
Test Loss: 3354440448.0, Test MAE: 50089.42578125


In [39]:
model.save('regression_model.h5')

h:\DLprj\DeeEnv\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
